In [3]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from pathlib import Path
import csv
import hvplot.pandas
import sqlalchemy as sql
import seaborn as sns
from matplotlib import pyplot as plt
import datetime as dt
from sqlalchemy import create_engine, inspect 
from tkinter import *
%matplotlib inline
from db_utils import get_debt_free_db_engine, get_transaction_for_user_year, get_account_for_user
from payment_projection import payment_projection

In [4]:
payment_projection(1, 500)

47106.490000000005


7.851081666666668

In [ ]:
# Create an engine to interact with the database
engine = get_debt_free_db_engine()

In [ ]:
transaction_2020 = get_transaction_for_user_year(engine, 1, '2020')

In [ ]:
transaction_2020['year'] = pd.DatetimeIndex(transaction_2020['transaction_date']).year

In [ ]:
transaction_2020['month'] = pd.DatetimeIndex(transaction_2020['transaction_date']).month

In [ ]:
transaction_2020.head()

In [ ]:
summary_transaction_2020 = pd.DataFrame(round(transaction_2020.groupby(['user_id', 'card','month'], as_index=False)[('amount')].sum(),2))
summary_transaction_2020

In [ ]:
#method for calculating how long to pay off debt given desired monthly amount
def payment_projection(user_id, monthly_amount):
    #get the amount on user's balance
    total_amount=summary_transaction_2020['amount'].iloc[user_id]
    #get number of months
    number_of_months=total_amount/monthly_amount
    # print("Num Months: " + str(number_of_months))
    #get number of years
    number_of_years=number_of_months/12
    # print("Num Years: " + str(number_of_years))
    #return the number of years for output to use in flask
    return number_of_years

In [ ]:
summary_transaction_2020.dtypes